In [1]:
from ltn_imp.automation.knowledge_base import KnowledgeBase
import torch
import pandas as pd

In [2]:
!poetry run poe download-medical-datasets

Poe => mkdir -p examples/medical/datasets
Poe => curl -L -o examples/medical/datasets/pima_indians_imputed.csv https://raw.githubusercontent.com/ChristelSirocchi/hybrid-ML/main/pima_indians_imputed.csv
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 33428  100 33428    0     0   598k      0 --:--:-- --:--:-- --:--:--  604k


In [3]:
import random
import numpy as np

seed = 42

# Set the random seed for reproducibility
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

In [4]:
from sklearn.model_selection import train_test_split
test_data = pd.read_csv('datasets/pima_indians_imputed.csv').astype(float)
x_train, x_test = train_test_split(test_data, test_size=0.5, random_state=seed)
x_train, x_val = train_test_split(x_train, test_size=0.1, random_state=seed)
x_train.to_csv('datasets/train.csv')
x_val.to_csv('datasets/val.csv')
x_test.to_csv('datasets/test.csv')

In [5]:
import torch
from sklearn.metrics import matthews_corrcoef

def predict(model, x):
    model.eval()  # Ensure the model is in evaluation mode
    with torch.no_grad():  # No need to track gradients
        # Ensure x is a tensor and has the right dtype
        if not isinstance(x, torch.Tensor):
            x = torch.tensor(x, dtype=torch.float32)
        elif x.dtype != torch.float32:
            x = x.float()
        
        # Forward pass through the model
        probs = model(x)
        
        # Apply binary classification threshold at 0.5
        preds = (probs > 0.5).float()
    return preds

def compute_metrics(model, data_loader):
    correct = 0
    total = 0
    
    true_positives = 0
    false_positives = 0
    false_negatives = 0
    true_negatives = 0
    
    all_true_labels = []
    all_predicted_labels = []
    
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():  # Disable gradient computation
        for data, labels in data_loader:
            # Ensure data and labels are the correct dtype
            if not isinstance(data, torch.Tensor):
                data = torch.tensor(data, dtype=torch.float32)
            elif data.dtype != torch.float32:
                data = data.float()
            
            if not isinstance(labels, torch.Tensor):
                labels = torch.tensor(labels, dtype=torch.float32)
            elif labels.dtype != torch.float32:
                labels = labels.float()
            
            # Get predictions
            preds = predict(model, data)
            
            # Squeeze predictions and labels to remove dimensions of size 1
            predicted_labels = preds.squeeze()
            true_labels = labels.squeeze()

            # Ensure the shapes match before comparison
            if predicted_labels.shape != true_labels.shape:
                true_labels = true_labels.view_as(predicted_labels)
            
            # Collect all predictions and true labels for MCC
            all_true_labels.extend(true_labels.cpu().numpy())
            all_predicted_labels.extend(predicted_labels.cpu().numpy())

            # Count correct predictions
            correct += (predicted_labels == true_labels).sum().item()
            total += true_labels.size(0)
            
            # Calculate TP, FP, FN, TN
            true_positives += ((predicted_labels == 1) & (true_labels == 1)).sum().item()
            false_positives += ((predicted_labels == 1) & (true_labels == 0)).sum().item()
            false_negatives += ((predicted_labels == 0) & (true_labels == 1)).sum().item()
            true_negatives += ((predicted_labels == 0) & (true_labels == 0)).sum().item()
    
    accuracy = correct / total if total > 0 else 0
    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    balanced_accuracy = 0.5 * (recall + (true_negatives / (true_negatives + false_positives) if (true_negatives + false_positives) > 0 else 0))
    mcc = matthews_corrcoef(all_true_labels, all_predicted_labels)
    tnr = true_negatives / (true_negatives + false_positives) if (true_negatives + false_positives) > 0 else 0  # True Negative Rate
    fpr = false_positives / (false_positives + true_negatives) if (false_positives + true_negatives) > 0 else 0  # False Positive Rate
    fnr = false_negatives / (false_negatives + true_positives) if (false_negatives + true_positives) > 0 else 0  # False Negative Rate
    tpr = recall  # True Positive Rate is the same as recall

    print(f"True Positives: {true_positives}, False Positives: {false_positives}, False Negatives: {false_negatives}, True Negatives: {true_negatives}, Total: {total}")
    print()
    print(f"A (Accuracy): {accuracy:.4f}")
    print(f"P (Precision): {precision:.4f}")
    print(f"R (Recall): {recall:.4f}")
    print(f"F1 (F1 Score): {f1_score:.4f}")
    print(f"BA (Balanced Accuracy): {balanced_accuracy:.4f}")
    print(f"MCC (Matthews Correlation Coefficient): {mcc:.4f}")
    print(f"TNR (True Negative Rate): {tnr:.4f}")
    print(f"FPR (False Positive Rate): {fpr:.4f}")
    print(f"FNR (False Negative Rate): {fnr:.4f}")
    print(f"TPR (True Positive Rate): {tpr:.4f}")

In [6]:
kb = KnowledgeBase("medical_config.yaml")

In [7]:
x, y = next(iter(kb.loaders[0]))
test_data = pd.DataFrame(x)
high = test_data[(test_data[5] > 29) & (test_data[1] > 125)]
low = test_data[(test_data[5] <= 25) & (test_data[1] <= 100)]

In [8]:
kb.predicates["Diabetic"](torch.tensor(high.values, dtype=torch.float32)).mean()

tensor(0.7317, grad_fn=<MeanBackward0>)

In [9]:
kb.predicates["Diabetic"](torch.tensor(low.values, dtype=torch.float32)).mean()

tensor(0.6648, grad_fn=<MeanBackward0>)

In [10]:
compute_metrics(kb.predicates["Diabetic"], kb.loaders[0])

True Positives: 124, False Positives: 221, False Negatives: 0, True Negatives: 0, Total: 345

A (Accuracy): 0.3594
P (Precision): 0.3594
R (Recall): 1.0000
F1 (F1 Score): 0.5288
BA (Balanced Accuracy): 0.5000
MCC (Matthews Correlation Coefficient): 0.0000
TNR (True Negative Rate): 0.0000
FPR (False Positive Rate): 1.0000
FNR (False Negative Rate): 0.0000
TPR (True Positive Rate): 1.0000


In [11]:
compute_metrics(kb.predicates["Diabetic"], kb.test_loaders[0])

True Positives: 130, False Positives: 254, False Negatives: 0, True Negatives: 0, Total: 384

A (Accuracy): 0.3385
P (Precision): 0.3385
R (Recall): 1.0000
F1 (F1 Score): 0.5058
BA (Balanced Accuracy): 0.5000
MCC (Matthews Correlation Coefficient): 0.0000
TNR (True Negative Rate): 0.0000
FPR (False Positive Rate): 1.0000
FNR (False Negative Rate): 0.0000
TPR (True Positive Rate): 1.0000


In [12]:
kb.optimize(num_epochs=100, log_steps=10, lr=0.001, early_stopping=True, patience=5)

['∀ person.(((y == diabetes) -> Diabetic(person)))', '∀ person.(((y == healthy) -> ~(Diabetic(person))))', '∀ person.((((person[BMI] < 26) & (person[Glucose] < 101)) -> ~(Diabetic(person))))', '∀ person.((((person[BMI] > 29) & (person[Glucose] > 125)) -> Diabetic(person)))', '∀ person.((((person[Glucose] > 143.5) & (person[DiabetesPedigreeFunction] > 0.32)) -> Diabetic(person)))', '∀ person.((((person[Glucose] > 143.5) & ((person[DiabetesPedigreeFunction] <= 0.32) & (person[BMI] <= 31.40))) -> ~(Diabetic(person))))', '∀ person.((((person[Glucose] > 143.5) & ((person[DiabetesPedigreeFunction] <= 0.32) & (person[BMI] > 31.40))) -> Diabetic(person)))', '∀ person.((((person[Glucose] <= 143.5) & ((person[Pregnancies] <= 7.5) & (person[BMI] <= 45.44))) -> ~(Diabetic(person))))', '∀ person.((((person[Glucose] <= 143.5) & ((person[Pregnancies] <= 7.5) & (person[BMI] > 45.44))) -> Diabetic(person)))', '∀ person.((((person[Glucose] <= 143.5) & ((person[Pregnancies] > 7.5) & (person[DiabetesPedig

In [13]:
compute_metrics(kb.predicates["Diabetic"], kb.loaders[0])

True Positives: 66, False Positives: 41, False Negatives: 58, True Negatives: 180, Total: 345

A (Accuracy): 0.7130
P (Precision): 0.6168
R (Recall): 0.5323
F1 (F1 Score): 0.5714
BA (Balanced Accuracy): 0.6734
MCC (Matthews Correlation Coefficient): 0.3597
TNR (True Negative Rate): 0.8145
FPR (False Positive Rate): 0.1855
FNR (False Negative Rate): 0.4677
TPR (True Positive Rate): 0.5323


In [14]:
compute_metrics(kb.predicates["Diabetic"], kb.test_loaders[0])

True Positives: 79, False Positives: 45, False Negatives: 51, True Negatives: 209, Total: 384

A (Accuracy): 0.7500
P (Precision): 0.6371
R (Recall): 0.6077
F1 (F1 Score): 0.6220
BA (Balanced Accuracy): 0.7153
MCC (Matthews Correlation Coefficient): 0.4357
TNR (True Negative Rate): 0.8228
FPR (False Positive Rate): 0.1772
FNR (False Negative Rate): 0.3923
TPR (True Positive Rate): 0.6077


In [15]:
kb.predicates["Diabetic"](torch.tensor(high.values, dtype=torch.float32)).mean()

tensor(0.5661, grad_fn=<MeanBackward0>)

In [16]:
kb.predicates["Diabetic"](torch.tensor(low.values, dtype=torch.float32)).mean()

tensor(0.0049, grad_fn=<MeanBackward0>)